In [187]:
%cd ~/shot-glass

import shot_glass.hifive.hifive_tools as hft
import shot_glass.hifive.hifive as hf
import shot_glass.hifive.hifive_test as hftest
from shot_glass.hifive.hifive import HiFive

from lunchbox.enforce import Enforce

import pandas as pd
from pandas import DataFrame

from itertools import takewhile, chain

/home/ubuntu/shot-glass


In [374]:
def collapse(hifive, source, target):
    src = source.indicator + '_id'
    tgt = target.indicator + '_id'
    col = f'{target.indicator}_x_{src}'
    data = hifive.map(src, col, lambda x: to_unique_list(x)).data
    data = data.groupby(tgt, as_index=False).first()
    data.drop(columns=[src], inplace=True)
    hifive.data = data
    return hifive

hi = HiFive()
hi.data = hftest.HiFiveTestBase().get_multi_data()
c = hf.HiFiveComponentType
hi = collapse(hi, c.VERTICES, c.EDGES)
# hi = collapse(hi, c.EDGES, c.FACES)
# hi = collapse(hi, c.FACES, c.ITEMS)
hi.data

,e_id,i_id,f_id,v_x,v_y,v_z,e_x_v_id
0,0,0,0,0.0,0.0,0.0,"[0, 1]"
1,1,0,0,1.0,1.0,1.0,"[1, 0]"
2,2,0,1,0.0,0.0,0.0,"[2, 3]"
3,3,0,1,1.0,1.0,1.0,"[3, 4]"
4,4,0,1,2.0,2.0,2.0,"[4, 5]"
5,5,0,1,3.0,3.0,3.0,"[5, 6]"
6,6,0,1,4.0,4.0,4.0,"[6, 2]"
7,7,0,2,0.0,0.0,0.0,"[7, 8]"
8,8,0,2,1.0,1.0,1.0,"[8, 9]"
9,9,0,2,2.0,2.0,2.0,"[9, 7]"


In [265]:
def to_vertex(vids, components):
    Enforce(len(set(vids)), '==', 1)
    Enforce(len(set(components)), '>=', 1)
    if isinstance(components, list):
        components = dict(zip(list('xyz'), components))
    return {vids[0]: components}

def to_edge(eids, vids):
    Enforce(len(set(eids)), '==', 1)
    Enforce(len(set(vids)), '==', 2)
    if isinstance(vids, list):
        vids = dict(zip(list('st'), vids))
    return {eids[0]: vids}

def to_face(fids, eids):
    Enforce(len(set(fids)), '==', 1)
    Enforce(len(set(eids)), '>=', 3)
    return {fids[0]: eids}

def to_item(iids, fids):
    Enforce(len(set(iids)), '==', 1)
    Enforce(len(set(fids)), '>=', 1)
    return {iids[0]: fids}

In [403]:
from dataclasses import dataclass

class Component:
    id = 0  # type: int
    parts = []  # type: List[float]

    @property
    def name(self):
        return self.__class__.__name__.lower()

    @property
    def indicator(self):
        return self.name[0]

    def validate(self, id_, parts):
        msg = f'ID must be an integer greater than -1. Given value: {id_}.'
        Enforce(id_, 'instance of', int, message=msg)
        Enforce(id_, '>=', 0, message=msg)

        msg = f'Parts must be a list. Given value: {parts}.'
        Enforce(parts, 'instance of', list, message=msg)

    def __init__(self, id_, parts):
        self.validate(id_, parts)
        self.id = id_
        self.parts = parts
        
    def __eq__(a, b):
        return (a.name, a.id, a.parts) == (b.name, b.id, b.parts)

    def __gt__(a, b):
        return a.id > b.id

    def __repr__(self):
        return f'{self.__class__.__name__}(id={self.id}, parts={self.parts})'
    
    @classmethod
    def from_series(cls, row, id_column, columns):
        id_ = row[id_column].tolist()[0]
        parts = []
        for col in columns:
            parts.append(row[col].tolist())
        parts = list(chain(*parts))
        return cls(id_, parts)
    
class Vertex(Component):
    pass

class Edge(Component):
    def validate(self, id_, parts):
        super().validate(id_, parts)
        msg = f'Parts must be a list of two unique integers. Given value: {parts}.'
        Enforce(len(parts), '==', 2, message=msg)
        Enforce(len(set(parts)), '==', 2, message=msg)
        Enforce(parts[0], 'instance of', int, message=msg)
        Enforce(parts[1], 'instance of', int, message=msg)
        
class Face(Component):
    def validate(self, id_, parts):
        super().validate(id_, parts)
        msg = f'Parts must be a list of unique integers. Given value: {parts}.'
        Enforce(len(set(parts)), '>=', 1, message=msg)
        Enforce(parts[0], 'instance of', int, message=msg)
        
class Item(Component):
    def validate(self, id_, parts):
        super().validate(id_, parts)
        msg = f'Parts must be a list of unique integers. Given value: {parts}.'
        Enforce(len(set(parts)), '>=', 1, message=msg)
        Enforce(parts[0], 'instance of', int, message=msg)

hi = HiFive()
hi.data = hftest.HiFiveTestBase().get_multi_data()
d = hi.data

grp = d.groupby('v_id', as_index=False)
d = grp.first().drop(columns=['v_id', 'v_x', 'v_y', 'v_z'])
d['v_x_id'] = grp.apply(lambda x: Vertex.from_series(x, 'v_id', ['v_x', 'v_y', 'v_z']))
# d['v_x_id'] = grp.apply(lambda x: Vertex(x.e_id.tolist()[0], [x.v_x, x.v_y, x.v_z]))[None]
d

# grp = d.groupby('e_id', as_index=False)
# d = grp.first().drop(columns=['v_id', 'e_id'])
# d['e_x_id'] = grp.apply(lambda x: Edge(x.e_id.tolist()[0], x.v_id.tolist()))[None]
d

   i_id  f_id  e_id  v_id  v_x  v_y  v_z
0     0     0     0     0  0.0  0.0  0.0
3     0     0     1     0  0.0  0.0  0.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
1     0     0     0     1  1.0  1.0  1.0
2     0     0     1     1  1.0  1.0  1.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
0     0     1     2     2  0.0  0.0  0.0
9     0     1     6     2  0.0  0.0  0.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
1     0     1     2     3  1.0  1.0  1.0
2     0     1     3     3  1.0  1.0  1.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
3     0     1     3     4  2.0  2.0  2.0
4     0     1     4     4  2.0  2.0  2.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
5     0     1     4     5  3.0  3.0  3.0
6     0     1     5     5  3.0  3.0  3.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
7     0     1     5     6  4.0  4.0  4.0
8     0     1     6     6  4.0  4.0  4.0
   i_id  f_id  e_id  v_id  v_x  v_y  v_z
0     0     2     7     7  0.0  0.0  0.0
5     0     2     9     7  0.0  0.0  0.0
   i_id  f_id  e

ValueError: Cannot set a DataFrame with multiple columns to the single column v_x_id

In [267]:
hi = HiFive()
hi.data = hftest.HiFiveTestBase().get_multi_data()

def func(row, source, target):
    src = to_unique_list(row[source])
    tgt = to_unique_list(row[target])
    return [to_edge(src, tgt)]

def collapse_row(row, source, target):
    src = source.indicator + '_id'
    tgt = target.indicator + '_id'
    src = to_unique_list(row[src])
    tgt = to_unique_list(row[tgt])
    return []

grp = hi.data.groupby('e_id', as_index=False)
grp.apply(lambda x: func(x, 'e_id', 'v_id'))

,e_id,None
0,0,"{0: {'s': 0, 't': 1}}"
1,1,"{1: {'s': 1, 't': 0}}"
2,2,"{2: {'s': 2, 't': 3}}"
3,3,"{3: {'s': 3, 't': 4}}"
4,4,"{4: {'s': 4, 't': 5}}"
5,5,"{5: {'s': 5, 't': 6}}"
6,6,"{6: {'s': 6, 't': 2}}"
7,7,"{7: {'s': 7, 't': 8}}"
8,8,"{8: {'s': 8, 't': 9}}"
9,9,"{9: {'s': 9, 't': 7}}"


In [126]:
def collapse(data, source, target='collapsed'):
    output = data \
        .groupby(source, as_index=False) \
        .apply(lambda x: [x.to_dict(orient='records')])
    output.rename(lambda x: target if x is None else x, axis=1, inplace=True)
    return output

def expand(data, column='collapsed'):
    output = x[column].apply(DataFrame).tolist()
    output = pd.concat(output).reset_index(drop=True)
    return output

def collapse(data, columns):
    temp = data \
        .groupby(columns, as_index=False) \
        .apply(lambda x: [x.to_dict(orient='list')])
    return temp[None]
    # temp = DataFrame(temp[None].tolist())
    # output = pd.concat([data[columns], temp], axis=1)
    # return output

# def expand(data, columns):
#     output = x[column].apply(DataFrame).tolist()
#     output = pd.concat(output).reset_index(drop=True)
#     return output

x = collapse(hi.data, 'e_id')
# x = expand(x)

x = hi.data
x = hi.data \
    .groupby('e_id', as_index=True) \
    .apply(lambda x: x.to_dict(orient='records'))

x = pd.concat(x.apply(DataFrame).tolist()).reset_index(drop=True)
x

# print( to_vertex([0, 0], [0, 1, 2]) )
# print( to_edge([0, 0], [0, 1])      )
# print( to_face([0, 0], [0, 1, 2])   )
# print( to_item([0, 0], [0])         )

hi.data \
    .groupby('f_id', as_index=False) \
    .apply(lambda x: [to_unique_list(x.e_id.tolist())])
    # .apply(lambda x: to_face(x.f_id.tolist(), x.e_id.tolist()))
    
def fold_component(data, source, target):
    output = data \
        .groupby(source, as_index=True) \
        .apply(lambda x: to_unique_list(x[target].tolist()))
    return output

def fold(data, component):
    c = hf.HiFiveComponentType
    comp = [c.VERTICES, c.EDGES, c.FACES, c.ITEMS]
    comp = takewhile(lambda x: x.order >= component.order, comp)
    comp = [x.indicator + '_id' for x in comp]
    comp = list(zip([None] + comp, comp))[1:]
    output = data
    for tgt, src in comp:
        grp = output \
            .groupby(src, as_index=False) \
            .aggregate(lambda x: x.tolist())
        output[src] = output[src].apply(lambda x: [x])
        output = output.applymap(lambda x: x[0] if len(x) == 1 else x)
    return output
               
# fold_component(hi.data, 'e_id', 'v_id')
x = fold(hi.data, hf.HiFiveComponentType.EDGES)
x
# x.groupby('f_id', as_index=False).aggregate(lambda x: x.tolist())

TypeError: unhashable type: 'list'

In [83]:
def to_unique_list(items):
    if isinstance(items, pd.Series):
        items = items.tolist()
    uniq = set(items)
    output = []
    for item in items:
        if item in uniq:
            output.append(item)
            uniq.remove(item)
        if len(uniq) == 0:
            break
    return output

x = [2, 1, 2, 1, 3, 2, 3]
to_unique_list(x)

[2, 1, 3]

In [31]:
grp = data.groupby('vid', as_index=False)
# grp.apply(lambda x: hft.fold_vertex(x.vid.tolist(), x[list('xyz')].tolist()))
# grp.apply(lambda x: [[x[col].tolist() for col in ['vid', 'x', 'y', 'z']]])
# x = grp.apply(lambda x: [x])
# x = x[None][1]
# x

def group_to_dict(group, columns=None):
    if columns is None:
        columns = group.columns

    output = {}
    for col in columns:
        output[col] = group[col].tolist()[0]
    return output

def fold(group, columns=None):
    x = group_to_dict(group, columns)
    vid = set(x['vid']).pop()
    
    return []

grp.apply(fold)

,vid,None
0,1,1
1,2,2
2,3,3
